# Pipeline de nettoyage de tweets pour LLM

Ce notebook présente un script complet de nettoyage de tweets.

Objectifs :
- Charger un fichier CSV de tweets (`free_tweet_export.csv`)
- Nettoyer le texte pour un usage NLP / LLM (suppression d'URLs, PII, normalisation…)
- Extraire les hashtags et détecter les retweets
- Supprimer les doublons et filtrer certains comptes
- Exporter un CSV propre (`free_tweet_export_cleaned.csv`) et afficher quelques statistiques


## Imports et définitions globales

On importe les bibliothèques nécessaires et on définit quelques **expressions régulières** pour :
- détecter les URLs
- reconnaître les retweets (`RT @...`)


In [19]:
import re, html, io, base64, pandas as pd
from pathlib import Path

URL_PAT     = re.compile(r'https?://\S+|www\.\S+', flags=re.IGNORECASE)
RT_PAT      = re.compile(r'^rt\s@', flags=re.IGNORECASE)

## Normalisation des sauts de ligne
### Fonction `_normalize_escaped_newlines`

Dans les CSV, les sauts de ligne peuvent parfois être stockés comme texte littéral `\\n`, `\\r`, `\\t`.

Cette fonction :
- remplace `\\n` par un vrai saut de ligne `\n`
- remplace `\\r` par `\r`
- remplace `\\t` par `\t`

Cela stabilise l'extraction des hashtags et le nettoyage du texte.


In [20]:
def _normalize_escaped_newlines(s: str) -> str:
    if not isinstance(s, str):
        return s
    return s.replace("\\r", "\r").replace("\\t", "\t").replace("\\n", "\n")


## Extraction des hashtags
### Fonction `extract_hashtags`

Cette fonction :
- reçoit un texte brut
- normalise les `\\n`, `\\t`, `\\r`
- extrait tous les hashtags avec une regex (`#[^\s#]+`)
- retourne une liste de hashtags (ex: `["#free", "#internet"]`)


In [21]:
def extract_hashtags(s: str):
    if not isinstance(s, str):
        return []
    text_norm = _normalize_escaped_newlines(s)
    return re.findall(r'#[^\s#]+', text_norm, flags=re.UNICODE)


## Nettoyage du texte pour LLM
### Fonction `clean_llm_text`

Cette fonction applique un **nettoyage léger** du texte pour LLM :

1. Normalisation des `\\n`, `\\t`, etc.
2. Décodage des entités HTML (`&amp;` → `&`, etc.)
3. Remplacement :
   - des URLs par `<URL>`
4. Aplatissement du texte :
   - suppression des sauts de ligne → espace
   - nettoyage des espaces en double
5. Suppression finale des marqueurs `<URL>`
6. Filet de sécurité : si des hashtags ont disparu après nettoyage, on les ré-append à la fin
7. Passage du texte en minuscules (via `casefold`, plus robuste que `lower`)


In [22]:
def clean_llm_text(raw: str) -> str:
    if not isinstance(raw, str):
        return raw

    # 1) Normalisation des séquences \n, \r, \t littérales
    cleaned = _normalize_escaped_newlines(raw)

    # 2) Décodage HTML
    cleaned = html.unescape(cleaned)

    # 3) Masquage PII + URLs
    cleaned = URL_PAT.sub('<URL>', cleaned)

    # 4) Normalisation des espaces / sauts de ligne
    cleaned = re.sub(r'[ \t]+', ' ', cleaned)
    cleaned = re.sub(r'\n+', ' ', cleaned).strip()

    # 5) Suppression du placeholder <URL> + re-nettoyage des espaces
    cleaned = re.sub(r'\s*<URL>\s*', ' ', cleaned)
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()

    # 6) Vérification : on réappend les hashtags éventuellement perdus
    raw_tags     = extract_hashtags(raw)
    cleaned_tags = extract_hashtags(cleaned)
    missing = [t for t in raw_tags if t not in cleaned_tags]
    if missing:
        cleaned += (' ' if not cleaned.endswith(' ') else '') + ' '.join(missing)

    # 7) Minuscule Unicode-safe
    return cleaned.casefold()


## Lecture robuste du CSV
### Fonction `read_csv_robust`

Cette fonction lit un fichier CSV en essayant deux encodages :
- d'abord `utf-8`
- puis `utf-8-sig` (souvent généré par Excel)

Elle utilise `sep=None` et `engine="python"` pour laisser pandas détecter le séparateur, et ignore les lignes problématiques (`on_bad_lines="skip"`).


In [23]:
def read_csv_robust(path: Path) -> pd.DataFrame:
    try:
        return pd.read_csv(path, sep=None, engine="python", encoding="utf-8", on_bad_lines="skip")
    except Exception:
        return pd.read_csv(path, sep=None, engine="python", encoding="utf-8-sig", on_bad_lines="skip")
